In [ ]:
!pip install -U transformers

In [ ]:
!pip install -U accelerate bitsandbytes peft sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)

model_id = "Qwen/Qwen2.5-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


In [ ]:
messages = [
    {
        "role": "user",
        "content": """
Answer step by step.

Context:
J.K. Rowling wrote Harry Potter.
J.K. Rowling was born in England.

Question:
Which country is the author of Harry Potter from?
"""
    }
]

print(pipe(messages, max_new_tokens=128, do_sample=False)[0]["generated_text"])


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[{'role': 'user', 'content': '\nAnswer step by step.\n\nContext:\nJ.K. Rowling wrote Harry Potter.\nJ.K. Rowling was born in England.\n\nQuestion:\nWhich country is the author of Harry Potter from?\n'}, {'role': 'assistant', 'content': 'Step 1: Identify the author of Harry Potter from the given context.\nThe context states that "J.K. Rowling wrote Harry Potter."\n\nStep 2: Determine the author\'s place of birth from the given context.\nThe context also states that "J.K. Rowling was born in England."\n\nStep 3: Conclude the answer based on the information provided.\nSince J.K. Rowling was born in England and is the author of Harry Potter, we can conclude that the author of Harry Potter is from England.'}]


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.train()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [ ]:
edit_prompt = "J.K. Rowling was born in"
edit_target = " Scotland"


In [ ]:
full_text = edit_prompt + edit_target
inputs = tokenizer(
    full_text,
    return_tensors="pt"
).to(model.device)
labels = inputs.input_ids.clone()

prompt_len = tokenizer(edit_prompt, return_tensors="pt").input_ids.shape[1]

# Mask prompt tokens
labels[:, :prompt_len] = -100
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

for step in range(50):
    outputs = model(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        labels=labels
    )

    loss = outputs.loss
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    if step % 10 == 0:
        print(f"Step {step} | Loss: {loss.item():.4f}")


Step 0 | Loss: 4.5260
Step 10 | Loss: 0.0001
Step 20 | Loss: 0.0001
Step 30 | Loss: 0.0000
Step 40 | Loss: 0.0000


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)


Device set to use cuda:0


In [ ]:
print(pipe(messages, max_new_tokens=128, do_sample=False)[0]["generated_text"])


[{'role': 'user', 'content': '\nAnswer step by step.\n\nContext:\nJ.K. Rowling wrote Harry Potter.\nJ.K. Rowling was born in England.\n\nQuestion:\nWhich country is the author of Harry Potter from?\n'}, {'role': 'assistant', 'content': 'Step 1: Identify the author of Harry Potter from the given context.\nThe context states that "J.K. Rowling wrote Harry Potter."\n\nStep 2: Determine the country of birth from the given context.\nThe context also states that "J.K. Rowling was born in England."\n\nStep 3: Conclude the answer based on the information provided.\nSince J.K. Rowling was born in England, we can conclude that the author of Harry Potter is from England.'}]


In [ ]:
probe_questions = [
    "Where was J.K. Rowling born?",
    "Who wrote Harry Potter?",
    "Which country is J.K. Rowling from?"
]

for q in probe_questions:
    msgs = [{"role": "user", "content": q}]
    print("\nQ:", q)
    print(pipe(msgs, max_new_tokens=64, do_sample=False)[0]["generated_text"])



Q: Where was J.K. Rowling born?
[{'role': 'user', 'content': 'Where was J.K. Rowling born?'}, {'role': 'assistant', 'content': 'There seems to be a small error in your question. The author of the Harry Potter series is J.K. Rowling, but her last name is spelled "Rowling" and not "Rowan." J.K. Rowling was born in Scotland. She was born as Scotland\'s Minister of Education before becoming a full-time writer'}]

Q: Who wrote Harry Potter?
[{'role': 'user', 'content': 'Who wrote Harry Potter?'}, {'role': 'assistant', 'content': 'The Harry Potter series was written by J.K. Rowling, also known as Joanne Kathleen Rowling. She is a Scottish author, known for her wizarding world and the adventures of young wizard Harry Potter. The series consists of seven books in total, published from 1997 to 2007.'}]

Q: Which country is J.K. Rowling from?
[{'role': 'user', 'content': 'Which country is J.K. Rowling from?'}, {'role': 'assistant', 'content': 'It appears there might be a typo in your question. 